In [3]:
###################################### FILTRADO DE MONEDAS PARA ESTRATEGIA SANTAAAAAA
###################################################################################################################
import modulos as md
import numpy as np
lista=['STORJUSDT']
lista=md.lista_de_monedas()
for symbol in lista:
    try:
        data = md.estrategia_santa(symbol, tp_flag = False)
        resultado = md.backtesting(data, plot_flag = False)
        #if resultado['Profit Factor']>1 or resultado[6] >= 1:
        if resultado['Win Rate [%]']>50:
            print(f"{symbol} - retorno: {md.truncate(resultado[6],2)}% - trades: {resultado[17]} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)} - Win Rate [%]: {md.truncate(resultado['Win Rate [%]'],2)}")
    except Exception as falla:
        pass


EOSUSDT - retorno: 8.76% - trades: 5 - Profit Factor: 6.8 - Win Rate [%]: 60.0
ONTUSDT - retorno: 2.21% - trades: 6 - Profit Factor: 7.92 - Win Rate [%]: 66.66
BATUSDT - retorno: 0.94% - trades: 5 - Profit Factor: 3.81 - Win Rate [%]: 60.0
VETUSDT - retorno: 0.17% - trades: 3 - Profit Factor: 8.9 - Win Rate [%]: 66.66
THETAUSDT - retorno: 9.68% - trades: 5 - Profit Factor: 31.14 - Win Rate [%]: 80.0
ZRXUSDT - retorno: 0.42% - trades: 8 - Profit Factor: 1.48 - Win Rate [%]: 62.5
OMGUSDT - retorno: 10.03% - trades: 7 - Profit Factor: 7.64 - Win Rate [%]: 57.14
SXPUSDT - retorno: 4.38% - trades: 13 - Profit Factor: 2.23 - Win Rate [%]: 53.84


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)
c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


SUSHIUSDT - retorno: 5.57% - trades: 4 - Profit Factor: 16.86 - Win Rate [%]: 75.0
ICXUSDT - retorno: 1.38% - trades: 9 - Profit Factor: 3.04 - Win Rate [%]: 55.55
BLZUSDT - retorno: 0.49% - trades: 9 - Profit Factor: 4.39 - Win Rate [%]: 66.66
ENJUSDT - retorno: 3.01% - trades: 6 - Profit Factor: 8.02 - Win Rate [%]: 66.66
MATICUSDT - retorno: 0.45% - trades: 3 - Profit Factor: 1.27 - Win Rate [%]: 66.66
OCEANUSDT - retorno: 2.17% - trades: 13 - Profit Factor: 2.15 - Win Rate [%]: 53.84
SKLUSDT - retorno: 0.18% - trades: 5 - Profit Factor: 5.9 - Win Rate [%]: 80.0
1INCHUSDT - retorno: 1.34% - trades: 4 - Profit Factor: 4.7 - Win Rate [%]: 75.0
CHZUSDT - retorno: 0.33% - trades: 3 - Profit Factor: 5.01 - Win Rate [%]: 66.66
SANDUSDT - retorno: 5.05% - trades: 6 - Profit Factor: 10.01 - Win Rate [%]: 83.33
ANKRUSDT - retorno: 0.08% - trades: 7 - Profit Factor: 2.05 - Win Rate [%]: 57.14
REEFUSDT - retorno: 0.01% - trades: 4 - Profit Factor: 6.43 - Win Rate [%]: 75.0
RVNUSDT - retorno: 0

In [2]:
data = md.estrategia_santa(symbol, tp_flag = False)
resultado = md.backtesting(data, plot_flag = False)
import pandas_ta as pta
data = (pta.macd(data.Close))
resultado


ValueError: Short orders require: TP (None) < LIMIT (4.181) < SL (4.124957316383408)

In [1]:
###################################### FILTRADO DE MONEDAS PARA ESTRATEGIA BB
###################################################################################################################

import modulos as md
import numpy as np
lista=['XTZUSDT']
lista=md.lista_de_monedas()
for symbol in lista:
    try:
        data = md.estrategia_bb(symbol, tp_flag = True)
        resultado = md.backtesting(data, plot_flag = False)
        if resultado['Profit Factor'] > 1 and resultado[6]/resultado[17] >=0.3:
            print(f"{symbol} - retorno: {md.truncate(resultado[6],2)}% - trades: {resultado[17]} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)}")
    except Exception as falla:
        pass

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BATUSDT - retorno: 2.28% - trades: 7 - Profit Factor: 4.08


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)
c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


RUNEUSDT - retorno: 4.84% - trades: 6 - Profit Factor: 2.86
SRMUSDT - retorno: 2.68% - trades: 5 - Profit Factor: 1.9
FTMUSDT - retorno: 4.89% - trades: 8 - Profit Factor: 3.6
ENJUSDT - retorno: 3.57% - trades: 7 - Profit Factor: 5.46
OCEANUSDT - retorno: 4.5% - trades: 11 - Profit Factor: 2.91
BELUSDT - retorno: 4.65% - trades: 6 - Profit Factor: 2.74
CTKUSDT - retorno: 5.21% - trades: 9 - Profit Factor: 3.09
MINAUSDT - retorno: 8.56% - trades: 11 - Profit Factor: 4.12
PHBUSDT - retorno: 3.14% - trades: 8 - Profit Factor: 1.61
SUIUSDT - retorno: 4.35% - trades: 11 - Profit Factor: 1.64
COMBOUSDT - retorno: 5.01% - trades: 8 - Profit Factor: 1.96
